In [10]:
import os
import cv2
import numpy as np
import pandas as pd
from glob import glob
from pathlib import Path
import re


# ====== USER CONFIGURATION ======
input_image_dir = r"N:\PROJ_TPE\TPE_20250525A01_e-5rps_conti_strain1"
input_label_dir = r"O:\LJJ202107\TPE_track_files\TPE_20250525A01_e-5rps_e-2fps_N=528_linked_500frames_20250527a_refined.pkl"  # folder with pkl files
output_base_dir = r"O:\LJJ202107\LJJ Jupyter Notebook Collection\Mask_RCNN\particle_track_cnn/dataset/train/"
image_ext = ".png"  # or ".jpg"
# =================================

os.makedirs(f"{output_base_dir}/images", exist_ok=True)
os.makedirs(f"{output_base_dir}/masks", exist_ok=True)

image_paths = sorted(glob(os.path.join(input_image_dir, f"*{image_ext}")))
image_paths = [path for path in image_paths if "Ib" not in os.path.basename(path)]
def extract_index(path):
    # Extract the base file name
    filename = os.path.basename(path)
    # Look for a pattern like "Ic_<number>" before the extension
    match = re.search(r"Ic_(\d+)", filename)
    return int(match.group(1)) if match else 0
# Sort the image_paths using the extracted index as key
sorted_image_paths = sorted(image_paths, key=extract_index)

F = pd.read_pickle(input_label_dir)

for frame in [100]:
    # Load image and coordinates
    image_path = sorted_image_paths[frame-1] #first image path is I_1
    print(image_path)
    image = cv2.imread(image_path)[:,:,0]

    # Save image copy to output
    cv2.imwrite(os.path.join(output_base_dir, "images", f"frame_{frame}{image_ext}"), image)

    df = F[F.frame==frame]
    # Create and save a mask for each disk
    for i, row in df.iterrows():
        x, y = int(row['x']), int(row['y'])
        ptype = (row['r_px']>45)*1
        mask = np.zeros(image.shape[:2], dtype=np.uint8)
        cv2.circle(mask, (x, y), int(row['r_px']), 255, -1)  # filled circle
        mask_filename = f"frame_{frame}_object_{int(row['particle'])}_type_{ptype}.png"
        cv2.imwrite(os.path.join(output_base_dir, "masks", mask_filename), mask)

print("✅ Dataset generation complete.")


N:\PROJ_TPE\TPE_20250525A01_e-5rps_conti_strain1\Ic_100.png
✅ Dataset generation complete.
